Возьму датасет, содержащий описание цветков ириса и их 
классификацию по сортам (Setosa, Versicolour, Virginica из в sklearn.datasets.load_iris.

In [26]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd

iris = load_iris()

Разделю данные на обучающую и тестирующую выборку в соотношение 70%/30%

In [27]:
x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.7)

С помощью метода LDA реализую классификацию сортов ириса на основании признаков датасета. Затем по метрикам из первого задания оценю эффективность классификатора.

In [28]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis()
lda.fit(x_train, y_train)

from sklearn.metrics import accuracy_score
pred = lda.predict(x_test)
print("Процент правильных ответов - ", round(accuracy_score(y_test, pred)*100, 2), "%")

Процент правильных ответов -  96.19 %


Процент правильных ответов достаточно высокий. Теперь посмотрим, матрицу ошибок.

In [29]:
from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(y_test, pred)
print(f'Первый вид ириса правильно определен - {matrix[0][0]} раз из {sum(matrix[0])}. В процентах - {round(matrix[0][0]/sum(matrix[0])*100, 2)} %')
print(f'Второй вид ириса правильно определен - {matrix[1][1]} раз из {sum(matrix[1])}. В процентах - {round(matrix[1][1]/sum(matrix[1])*100, 2)} %')
print(f'Третий вид ириса правильно определен - {matrix[2][2]} раз из {sum(matrix[2])}. В процентах - {round(matrix[2][2]/sum(matrix[2])*100, 2)} %')

Первый вид ириса правильно определен - 32 раз из 32. В процентах - 100.0 %
Второй вид ириса правильно определен - 40 раз из 41. В процентах - 97.56 %
Третий вид ириса правильно определен - 29 раз из 32. В процентах - 90.62 %


Как можно видеть, классификатор выдал очень хорошие результаты. И при каждом обучении доля правильных ответов в любом классе выше 90%

Сравню метод LDA из sklearn.discriminant_analysis и реализацию из Sem3.ipynb.
Для этого скопирую сюда реализацию из sem3.ipynb

In [30]:
import numpy as np

def LDA_dimensionality(X, y, k):
    '''
    X - набор данных, y - метка, k - целевой размер
    '''
    label_ = list(set(y))

    X_classify = {}

    for label in label_:
        X1 = np.array([X[i] for i in range(len(X)) if y[i] == label])
        X_classify[label] = X1

    mju = np.mean(X, axis=0)
    mju_classify = {}

    for label in label_:
        mju1 = np.mean(X_classify[label], axis=0)
        mju_classify[label] = mju1

    #St = np.dot((X - mju).T, X - mju)

    Sw = np.zeros((len(mju), len(mju)))  # Вычислить матрицу внутриклассовой дивергенции
    for i in label_:
        Sw += np.dot((X_classify[i] - mju_classify[i]).T,
                     X_classify[i] - mju_classify[i])

    # Sb=St-Sw

    Sb = np.zeros((len(mju), len(mju)))  # Вычислить матрицу внутриклассовой дивергенции
    for i in label_:
        Sb += len(X_classify[i]) * np.dot((mju_classify[i] - mju).reshape(
            (len(mju), 1)), (mju_classify[i] - mju).reshape((1, len(mju))))

    eig_vals, eig_vecs = np.linalg.eig(
        np.linalg.inv(Sw).dot(Sb))  # Вычислить собственное значение и собственную матрицу Sw-1 * Sb

    sorted_indices = np.argsort(eig_vals)
    topk_eig_vecs = eig_vecs[:, sorted_indices[:-k - 1:-1]]  # Извлекаем первые k векторов признаков
    return topk_eig_vecs

 Проведу классификацию

In [31]:
import numpy as np
W = LDA_dimensionality(x_train, y_train, 4)
X_new = np.dot((x_test), W)
#print("Процент правильных ответов - ", round(accuracy_score(y_test, X_new)*100, 2), "%")

In [32]:
matrix = confusion_matrix(y_test, pred)
print(f'Первый вид ириса правильно определен - {matrix[0][0]} раз из {sum(matrix[0])}. В процентах - {round(matrix[0][0]/sum(matrix[0])*100, 2)} %')
print(f'Второй вид ириса правильно определен - {matrix[1][1]} раз из {sum(matrix[1])}. В процентах - {round(matrix[1][1]/sum(matrix[1])*100, 2)} %')
print(f'Третий вид ириса правильно определен - {matrix[2][2]} раз из {sum(matrix[2])}. В процентах - {round(matrix[2][2]/sum(matrix[2])*100, 2)} %')

Первый вид ириса правильно определен - 32 раз из 32. В процентах - 100.0 %
Второй вид ириса правильно определен - 40 раз из 41. В процентах - 97.56 %
Третий вид ириса правильно определен - 29 раз из 32. В процентах - 90.62 %


Из результатов работы можно видеть, что реализация библиотеки sklearn и реализация из файла Sem3.ipynb выдают

Рассмотрите документацию метода LDA и измените параметры классификатора таким образом, чтобы алгоритм работал эффективнее
(например, поменять параметр solver). Какие параметры классификатора сильнее сказываются на конечном результате? Чем это может
быть объяснимо?

In [42]:
lda = LinearDiscriminantAnalysis(solver = 'lsqr')
lda.fit(x_train, y_train)

pred = lda.predict(x_test)
print("Процент правильных ответов - ", round(accuracy_score(y_test, pred)*100, 2), "%")

Процент правильных ответов -  96.19 %
